In [251]:
#this script take the news data from the google and historical/current stock data from the yahoo,
#then label the DS based on the open/close price. if closing is more than opening it's 1 else 0.

import pandas as pd
from datetime import date
from pandas_datareader import data
import datetime as datetime
 
#for labelling
def cal_label(row):
    if row['Open'] == row['Close']:
        val = 1
    elif row['Open'] > row['Close']:
        val = 0
    else:
        val = 1
    return val

#dateset creation for ticker
def companyTickerprice(company,ticker):
    start_date = (datetime.datetime.now() - datetime.timedelta(3800)).strftime("%Y-%m-%d")
    end_date = date.today()

    panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
   
    stock_price_ds=panel_data[['Open','Close']]
    stock_price_ds['Label']=stock_price_ds.apply(cal_label, axis=1)
    stock_price_ds['Datecol']=stock_price_ds.index.to_series()
    stock_price_ds['Datecol']=pd.to_datetime(stock_price_ds['Datecol']).dt.strftime('%m-%d-%Y')
    stock_price_ds['Company']=company
    
    return stock_price_ds    

# Function grabs the rss feed headlines (titles) and returns them as a list
#search by company name
def getHeadlinesbycompanyName( company, rss_url):
    companyName=[]
    headlines = []
    dateposted=[]
    description=[]
    
    d={'Company':companyName,'Title':headlines,'Datecol':dateposted,'Description':description}
  
    feed = parseRSS( rss_url )
  
    for newsitem in feed['items']:        
     headlines.append(newsitem['title']) 
     companyName.append(company)
     dateposted.append(newsitem['published'])       
        #description.append(newsitem['summary'])
       # print(newsitem['summary'])
     description.append(newsitem['link'])       
    
    df=pd.DataFrame(d)  
    df['Datecol']=pd.to_datetime(df['Datecol'], format = "%a, %d %b %Y %H:%M:%S %Z").apply(lambda x: x.strftime('%m-%d-%Y'))
    return df
 
#search by company ticker
def getHeadlinesbyticker( company, rss_url):
    companyName=[]
    headlines = []
    dateposted=[]
    description=[]
    
    d={'Company':companyName,'Title':headlines,'Datecol':dateposted,'Description':description}
  
    feed = parseRSS( rss_url )
   
    for newsitem in feed['items']:        
        if(company.lower() in newsitem['title'].lower()):
         headlines.append(newsitem['title']) 
         companyName.append(company)
         dateposted.append(newsitem['published'])       
        #description.append(newsitem['summary'])
       # print(newsitem['summary'])
         description.append(newsitem['link'])       
    
    df=pd.DataFrame(d)  
    df['Datecol']=pd.to_datetime(df['Datecol'], format = "%a, %d %b %Y %H:%M:%S %Z").apply(lambda x: x.strftime('%m-%d-%Y'))
    return df    
    
# List of RSS feeds that we will fetch and combine
newsurls = {
    #'googlenews':       'https://news.google.com/rss/search?q=aapl',
    'yahoonews':        'http://finance.yahoo.com/rss/headline?s=aapl'
}

# List of tickers that we will fetch and combine
ticker = {'Apple':'AAPL','Alphabet':'GOOGL','Amazon':'AMZN','Microsoft':'MSFT','Cisco':'CSCO','Ares':'ARCC',
         'Apollo':'AINV','Hercules':'HTGC','Blackrock':'BKCC',
         'American Airlines':'AAL','Southwest':'LUV','JetBlue':'JBLU',
         }
tickerComp = {'Delta':'DAL','Hilton':'HLT','Marriot':'MAR','Hyatt':'H','Wyndham':'WH'}
#ticker={'Apple':'AAPL'}   



In [249]:
#creating the dataset in csv, pulling the data from google news. this function will pass the ticker of company
for key,ticker in ticker.items():
    dsNews=getHeadlinesbyticker(key,'https://news.google.com/rss/search?q='+ticker)  
    dsStockPrice=companyTickerprice(key,ticker)  
    dsLabelled = pd.merge(dsStockPrice, dsNews, on='Datecol', how='inner')
    dsLabelled.rename(columns = {'Company_x':'Firm','Datecol':'Date','Title':'Headlines'}, inplace = True) 
    dsLabelledFinal = dsLabelled[['Firm','Date','Headlines','Label','Description','Open','Close']]
    dsLabelledFinal.to_csv('C:\\Dev\\stock_news_data\\Raw\\Ds_{}.csv'.format(key+"_"+date.today().strftime("%m-%d-%Y")))
    print(dsLabelledFinal.head(2))

#creating the dataset in csv, pulling the data from google news. this function will pass the name of company

for key,ticker in tickerComp.items():
    dsNews=getHeadlinesbycompanyName(key,'https://news.google.com/rss/search?q='+key)  
    dsStockPrice=companyTickerprice(key,ticker)  
    dsLabelled = pd.merge(dsStockPrice, dsNews, on='Datecol', how='inner')
    dsLabelled.rename(columns = {'Company_x':'Firm','Datecol':'Date','Title':'Headlines'}, inplace = True) 
    dsLabelledFinal = dsLabelled[['Firm','Date','Headlines','Label','Description','Open','Close']]
    dsLabelledFinal.to_csv('C:\\Dev\\stock_news_data\\Raw\\Ds_{}.csv'.format(key+"_"+date.today().strftime("%m-%d-%Y")))
    print(dsLabelledFinal.head(2))

c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

    Firm        Date                                          Headlines  \
0  Apple  03-19-2020  Apple (AAPL) Stock Sinks As Market Gains: What...   
1  Apple  04-13-2020  Apple Reportedly Planning Major iPhone Design ...   

   Label                                        Description        Open  \
0      0  https://finance.yahoo.com/news/apple-aapl-stoc...  247.389999   
1      1  https://www.benzinga.com/news/20/04/15789683/a...  268.309998   

        Close  
0  244.779999  
1  273.250000  


c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

       Firm        Date                                          Headlines  \
0  Alphabet  02-03-2020  Alphabet shares slide despite earnings beat - ...   
1  Alphabet  04-06-2020  Alphabet (GOOGL) Tracks Coronavirus Cases Usin...   

   Label                                        Description         Open  \
0      1  https://www.cnbc.com/2020/02/03/alphabet-googl...  1461.650024   
1      1  https://finance.yahoo.com/news/alphabet-googl-...  1133.000000   

         Close  
0  1482.599976  
1  1183.189941  


c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

...

c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

      Firm        Date                                          Headlines  \
0  Marriot  09-15-2014  Marriot hotels are pressing visitors to tip th...   
1  Marriot  10-12-2015  Marriott Brands -- All the Hotels Under the Ma...   

   Label                                        Description       Open  \
0      1  https://nypost.com/2014/09/15/marriot-hotels-a...  70.540001   
1      0  https://www.fool.com/investing/general/2015/10...  76.510002   

       Close  
0  70.709999  
1  76.410004  


c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

    Firm        Date                                          Headlines  \
0  Hyatt  11-12-2019  Hyatt Joins Other Big Hotel Chains by Pledging...   
1  Hyatt  12-17-2019  Hyatt Announces Plans for First Property in Fi...   

   Label                                        Description       Open  \
0      0  https://skift.com/2019/11/12/hyatt-joins-other...  76.519997   
1      1  https://www.businesswire.com/news/home/2019121...  84.779999   

       Close  
0  76.160004  
1  85.349998  
      Firm        Date                                          Headlines  \
0  Wyndham  12-02-2019  Bluebonnet Wyndham hotel to be rebranded - The...   
1  Wyndham  12-03-2019  Wyndham Hotels Promotes Finance Exec to CFO Am...   

   Label                                        Description       Open  \
0      0  https://www.theadvocate.com/baton_rouge/news/b...  57.849998   
1      0  https://www.wsj.com/articles/wyndham-hotels-pr...  57.369999   

       Close  
0  57.689999  
1  56.299999  


c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\amank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer